## 输出层的设计

### 神经网络两个具体问题

&emsp;上面一小节上我们在最后的输出层使用的是恒等函数，也就是直接将最后一层的信号量通过加权计算后的数据直接在输出层输出来。这里我们就要重点来看看输出层到底需要怎么来输出。  
&emsp;一般神经网络输出层有两种分类，这个分类是根据实际网络需要解决的具体问题而定：
- 分类问题：一般选用softmax函数
    - 判断输入的数据是属于哪个类别，比如是猫还是狗
- 回归问题：一般选用恒等函数
    - 用来预测具体数值问题，比如预测温度，体重等具体数值
    
### 恒等函数和 softmax 函数

#### 恒等函数
&emsp;上面一小节上我们在最后的输出层使用的是恒等函数，也就是直接将最后一层的信号量通过加权计算后的数据直接在输出层输出来。所以这里就可以直接表示如下图：

![恒等函数](imgs/16.jpg)

#### softmax 函数

&emsp;softmax函数其实没有那么神秘，其实就是对最后的值计算一个指数，然后将所有的输出层求和，在计算各个输出所占的比例。计算公式如下：  
$y_k =\frac{exp(a_k)}{\sum^n_{i=1}{exp(a_i)}} $

- $a_k$:表示第k个输出的输入信号
- 分子:表示$a_k$的指数函数
- 分母:表示所有信号的指数函数的和

这里通过一个例子来理解,


In [3]:
import numpy as np

# 倒数第二层的输入加权信号为
a = np.array([0.1,-0.2,0.3])
# 对输入信号进行指数函数计算
z = np.exp(a)
print(z)
# 计算总和
z_sum = np.sum(z)
print(z_sum)
#计算softmax值
#其实就是计算对于输出所占的比例
y = z/z_sum
print(y)
# y_sum这个值是1
y_sum = np.sum(y)
print(y_sum)

[1.10517092 0.81873075 1.34985881]
3.2737604787296326
[0.33758454 0.25008878 0.41232669]
1.0


In [4]:
# softmax函数可以归纳如下
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y

### softmax函数需要注意的点

&ensp;&ensp;上面我使用softmax来分类,里面有一个最大的特点就是使用指数函数.指数函数这里在数学计算中可能没有什么特别的地方,但是到了计算机就有不同了,因为计算机对于数据的大小表示是由范围限制的.比如:$e^{10}$与$e^{1000}$,计算机的表示是不同的.后者可能表示为无穷大，出现溢出的情况。

&ensp;&ensp;为了让数据的输入在一个正常的范围,让计算能够能够正常显示,下面我们对这个公式做一个变换.这样即使输入的数据是一个比较大的值，我们也可让计算机正常的表示而且又不破坏数据的正确性。

具体的公式如下：

![](imgs/17.jpg)

上面的公式利用以下的特性：
- 分子分母同时乘以常数，结果不会改变
- 利用指数函数和对数函数的特性
最终想表达的意思就是：sofrmax函数中，加上（或者减去）
某个常数并不会改变运算的结果。
利用该特性，防止数据溢出，这里可以减去某个常数（通常是所有数据中的最大值）。

In [6]:
import numpy as np
a= np.array([10001,10010,9990])
print(np.exp(a)/np.sum(np.exp(a)))

c = np.max(a)
b= a-c
print(b)
print(np.exp(b)/np.sum(np.exp(b)))

[nan nan nan]
[ -9   0 -20]
[1.23394576e-04 9.99876603e-01 2.06089928e-09]


d:\program files\python35\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
d:\program files\python35\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# softmax 函数的改进

def softmax(a):
    c= np.max(a)
    b = a - c # 防止溢出
    exp_a = np.exp(b)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    return y

### softmax 函数的特征

从上面的实例中我们可以看出softmax有如下特征：
- softmax值的总和是1
- softmax输出的每个值其实就是该值输出的概率
    -例如y[0] = 0.18, 就是第0个类别的概率为18%
- 各个元素之间的大小关系也不会改变。这是因为指数函数（y = exp(x)）是单调递增函数
- 神经网络只把输出值最大的神经元所对应的类别作为识别结果。

由于这些因素才导致我们在分类经常在输出层采用softmax函数来处理。

### 输出层的神经元数量设定

输出层的神经元数量设定需要根据具体的情况来定，说的简单点就是我们需要分辨几类，就设定几个输出层。
比如我们需要分辨：猫，狗，猪，羊，这里我们就需要设定4个输出
再比如需要识别数字：0-9，这里我们就需要设定10个输出.
